<a href="https://colab.research.google.com/github/alanmejiaa/Aprendizale-Automatico/blob/main/Tarea%20%236.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 6 (10 puntos)**
* Investigar sobre algoritmos supervisados que puedan aplicarse a tu problema. Da un panorama sobre el modelo matemático que emplean y explica por qué conviene utilizarlo

* Investigar sobre métricas para analizar los errores de tus datos y discutirla (generalmente se usan MAE, MAPE, RMSE, MSE...)
* Aplicar al menos un algoritmo supervisado a tus datos para predecir alguna variable (puede ser interpolación o extrapolación)
* Actualiza literatura relacionada, metodología, resultados y discusión en tu artículo
* Sube el código de tu tarea, los archivos de Latex y el PDF del artículo en tu repositorio, claramente diferenciados

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from prettytable import PrettyTable
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('skudata.csv')

Tomamos solo las variables de ínteres

In [6]:
df = pd.read_csv("skudata.csv")
df = df[['L13UnitsTotal', '_28', 'AVG_BASE_COST58', 'Busquedas', 'Instock', 'CR']]
df.head

<bound method NDFrame.head of      L13UnitsTotal  _28  AVG_BASE_COST58  Busquedas    Instock      CR
0          1097507  125               38     991269     0.9790  0.4875
1           891483   22                3     967704     0.9625  0.0506
2           667941  129               58     725171     0.9970  0.3612
3           544646   22                4     889493     0.8800  0.0484
4           477564   67               29     872521     0.9646  0.2278
..             ...  ...              ...        ...        ...     ...
295          63061  179               88     203511  9407.0000  0.4654
296          62929    5                2     645412  9457.0000  0.0130
297          62929  199               77     292465  9075.0000  0.6766
298          62851   51               14     869680  9860.0000  0.1479
299          62810  105               41     571263  9020.0000  0.3675

[300 rows x 6 columns]>

In [7]:
print(df.iloc[:10].to_latex(index=False))

\begin{tabular}{rrrrrr}
\toprule
 L13UnitsTotal &  \_28 &  AVG\_BASE\_COST58 &  Busquedas &  Instock &     CR \\
\midrule
       1097507 &  125 &               38 &     991269 &   0.9790 & 0.4875 \\
        891483 &   22 &                3 &     967704 &   0.9625 & 0.0506 \\
        667941 &  129 &               58 &     725171 &   0.9970 & 0.3612 \\
        544646 &   22 &                4 &     889493 &   0.8800 & 0.0484 \\
        477564 &   67 &               29 &     872521 &   0.9646 & 0.2278 \\
        439706 &  215 &               74 &     869196 &   0.9166 & 0.7310 \\
        420348 &   32 &               10 &     259422 &   0.9423 & 0.0704 \\
        408229 &   79 &               31 &     917866 &   0.9546 & 0.2765 \\
        392437 &   22 &                3 &     884690 &   0.9060 & 0.0638 \\
        385173 &   79 &               12 &     764001 &   0.9777 & 0.2212 \\
\bottomrule
\end{tabular}



Se establece el 80% de los datos para el train y 20% para el test

In [9]:
test1_df = df.tail(60) #20%
test_df = test1_df.drop("L13UnitsTotal", axis=1)
test_df

,_28,AVG_BASE_COST58,Busquedas,Instock,CR
240,149,65,714753,9743.0,0.3725
241,2799,1052,698803,9345.0,8.3970
242,27,11,658838,9440.0,0.0648
243,165,56,799878,8855.0,0.6105
244,95,33,859530,9871.0,0.3610
245,105,40,454683,9049.0,0.2730
246,2899,1037,661872,9143.0,6.9576
247,2999,1234,673539,9762.0,11.3962
248,59,17,911417,9952.0,0.1475
249,199,99,406925,9486.0,0.4577


In [10]:
skudata_df = df.drop(test_df.index)
skudata_df

,L13UnitsTotal,_28,AVG_BASE_COST58,Busquedas,Instock,CR
0,1097507,125,38,991269,0.9790,0.4875
1,891483,22,3,967704,0.9625,0.0506
2,667941,129,58,725171,0.9970,0.3612
3,544646,22,4,889493,0.8800,0.0484
4,477564,67,29,872521,0.9646,0.2278
...,...,...,...,...,...,...
235,74871,159,47,443753,9245.0000,0.4929
236,74739,56,26,427766,8931.0000,0.1400
237,74655,99,40,558569,8935.0000,0.3465
238,74394,189,38,1008337,9110.0000,0.5481


Haremos pruebas para los siguientes modelos:
* Logistic Regression
* KNN or k-Nearest Neighbors
* Support Vector Machines
* Naive Bayes classifier
* Decision Tree
* Random Forrest
* Perceptron
* Artificial neural network
* RVM or Relevance Vector Machine

In [11]:
X_skudata = skudata_df.drop("L13UnitsTotal", axis=1)
Y_skudata = skudata_df["L13UnitsTotal"]
X_test  = test_df
X_skudata.shape, Y_skudata.shape, X_test.shape

((240, 5), (240,), (60, 5))

In [12]:
logreg = LogisticRegression()
logreg.fit(X_skudata, Y_skudata)
Y_predLR = logreg.predict(X_test)
acc_log = round(logreg.score(X_skudata, Y_skudata) * 100, 2)
acc_log

23.75

In [13]:
svc = SVC()
svc.fit(X_skudata, Y_skudata)
Y_predSVM = svc.predict(X_test)
acc_svc = round(svc.score(X_skudata, Y_skudata) * 100, 2)
acc_svc

100.0

In [14]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_skudata, Y_skudata)
Y_predKNN = knn.predict(X_test)
acc_knn = round(knn.score(X_skudata, Y_skudata) * 100, 2)
acc_knn

29.58

In [15]:
gaussian = GaussianNB()
gaussian.fit(X_skudata, Y_skudata)
Y_predGNB = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_skudata, Y_skudata) * 100, 2)
acc_gaussian

100.0

In [16]:
perceptron = Perceptron()
perceptron.fit(X_skudata, Y_skudata)
Y_predP = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_skudata, Y_skudata) * 100, 2)
acc_perceptron

0.42

In [17]:
linear_svc = LinearSVC()
linear_svc.fit(X_skudata, Y_skudata)
Y_predLSVC = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_skudata, Y_skudata) * 100, 2)
acc_linear_svc

0.83

In [18]:
sgd = SGDClassifier()
sgd.fit(X_skudata, Y_skudata)
Y_predSGD = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_skudata, Y_skudata) * 100, 2)
acc_sgd

0.42

In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_skudata, Y_skudata)
Y_predDT = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_skudata, Y_skudata) * 100, 2)
acc_decision_tree

100.0

In [20]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_skudata, Y_skudata)
Y_predRF = random_forest.predict(X_test)
random_forest.score(X_skudata, Y_skudata)
acc_random_forest = round(random_forest.score(X_skudata, Y_skudata) * 100, 2)
acc_random_forest

100.0

Revisando cada uno de los modelos:

In [21]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,Support Vector Machines,100.00
3,Random Forest,100.00
4,Naive Bayes,100.00
8,Decision Tree,100.00
1,KNN,29.58
2,Logistic Regression,23.75
7,Linear SVC,0.83
5,Perceptron,0.42
6,Stochastic Gradient Decent,0.42


In [22]:
models1 = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})

In [23]:
print(models.sort_values(by='Score', ascending=False).to_latex(index=False))

\begin{tabular}{lr}
\toprule
                     Model &  Score \\
\midrule
   Support Vector Machines & 100.00 \\
             Random Forest & 100.00 \\
               Naive Bayes & 100.00 \\
             Decision Tree & 100.00 \\
                       KNN &  29.58 \\
       Logistic Regression &  23.75 \\
                Linear SVC &   0.83 \\
                Perceptron &   0.42 \\
Stochastic Gradient Decent &   0.42 \\
\bottomrule
\end{tabular}



Podemos concluir que el mejor modelo es el de Support Vector Machines con un 100% de precisión.